In [1]:
from pulp import LpProblem, LpMaximize, LpVariable, LpMinimize

### Прямая задача ЛП - максимизация прибыли

In [2]:
# Коэффициенты
a = 4
b = 4
c = 32

# Цены
p1 = 6 * b + 12
p2 = 5 * b + 22
p3 = c


# Матрица расходов
A = [
   # A  B  C
    [1, 3, a], # I
    [6, 5, 2] # II
]


# Запасы
lim = [3000, 3320]

In [3]:
model = LpProblem(name="resource-allocation", sense = LpMaximize)
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)

#Целевая функция
model += p1 * x1 + p2 * x2 + p3 * x3

# Ограничения
model += A[0][0] * x1 + A[0][1] * x2 + A[0][2] * x3 <= lim[0]
model += A[1][0] * x1 + A[1][1] * x2 + A[1][2] * x3 <= lim[1]

In [4]:
model

resource-allocation:
MAXIMIZE
36*x1 + 42*x2 + 32*x3 + 0
SUBJECT TO
_C1: x1 + 3 x2 + 4 x3 <= 3000

_C2: 6 x1 + 5 x2 + 2 x3 <= 3320

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

In [5]:
model.solve()

1

In [6]:
r1 = sum(map(lambda x, y: x.value() * y, model.variables(), A[0]))
r2 = sum(map(lambda x, y: x.value() * y, model.variables(), A[1]))

print('Реальный расход ресурсов =', (r1, r2))

Реальный расход ресурсов = (3000.0, 3320.0)


In [7]:
d1 = abs(r1 - lim[0])
d2 = abs(r2 - lim[1])

print(f"Дефицит ресурсов = ", (d1, d2))

Дефицит ресурсов =  (0.0, 0.0)


In [8]:
X = list(map(lambda x: int(x.value()), model.variables()))
res = f'X* = ({";".join(map(str, X))}); F_max = {int(model.objective.value())}'
res

'X* = (0;520;360); F_max = 33360'

### Двойственная задача ЛП - минимизация стоимости ресурсов

In [14]:
d_model = LpProblem(name="resource-allocation", sense=LpMinimize)

# Ограничения прямой задачи делаем переменными
y1 = LpVariable(name="y1",  lowBound=0)
y2 = LpVariable(name="y2",  lowBound=0)

In [15]:
# Целевая функция
# Минимизировать целевую функцию двойственных переменных
d_model += lim[0] * y1 + lim[1] * y2

# Ограничения (являются переменными в прямой)
d_model += A[0][0] * y1 + A[1][0] * y2 >= p1
d_model += A[0][1] * y1 + A[1][1] * y2 >= p2
d_model += A[0][2] * y1 + A[1][2] * y2 >= p3

In [16]:
d_model

resource-allocation:
MINIMIZE
3000*y1 + 3320*y2 + 0
SUBJECT TO
_C1: y1 + 6 y2 >= 36

_C2: 3 y1 + 5 y2 >= 42

_C3: 4 y1 + 2 y2 >= 32

VARIABLES
y1 Continuous
y2 Continuous

In [12]:
d_model.solve()

1

In [13]:
Y = list(map(lambda x: x.value(), d_model.variables()))
res = f'Y* = ({";".join(map(str, Y))}); F_min = {d_model.objective.value()}'
res

'Y* = (5.4285714;5.1428571); F_min = 33359.999771999996'

### Вывод
Мы нашли оптимальные цены для ресурсов, имевшие ограничения на запасы, при этом прибыль осталась такой же, как и в прямой задаче.